# Fitbit Food Data

This notebook walks through how to pull sleep, food & calories out data using the Fitbit API.

In [174]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [175]:
import sys
sys.path.append('../src/')
import plotutils
import fitbitqueries as fbq

In [194]:
import fitbit
import os
import getpass
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Connect to fitbit API

In [177]:
os.environ['client_id'] = '22BC6L'
os.environ['client_secret'] = 'e8348b3b97bba4db4fb142928cedfc31'

In [178]:
!/Users/hasannagib/opt/anaconda3/envs/fitbit/bin/python ../src/generate_tokens.py $client_id $client_secret

[02/Feb/2020:14:52:19] ENGINE Listening for SIGTERM.
[02/Feb/2020:14:52:19] ENGINE Listening for SIGHUP.
[02/Feb/2020:14:52:19] ENGINE Listening for SIGUSR1.
[02/Feb/2020:14:52:19] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[02/Feb/2020:14:52:19] ENGINE Started monitor thread 'Autoreloader'.
[02/Feb/2020:14:52:20] ENGINE Serving on http://127.0.0.1:8080
[02/Feb/2020:14:52:20] ENGINE Bus STARTED
127.0.0.1 - - [02/Feb/2020:14:52:31] "GET /?code=e64d7c12f67e17e44d1d83216593e353a0f5589d&state=wjnwuqtpyEnpcW09bq67GZjLxcsJQb HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
[02/Feb/2020:14:52:32] ENGINE Bus STOPPING
[02/Feb/2020:14:52:32] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[02/Feb/2020:14:52:32] ENGINE Stopped thread 'Autoreloader'.
[02/Feb/2020:14:52:32] ENGINE Bus STOPPED
[02/Feb/2020:14:52:32] ENGIN

In [179]:
with open('../data/access_token.txt','r') as f:
    os.environ['fitbit_access_token'] = f.read()

with open('../data/refresh_token.txt','r') as f:
    os.environ['fitbit_refresh_token'] = f.read()
    
auth_client = fitbit.Fitbit(
    os.environ['client_id'], 
    os.environ['client_secret'],
    os.environ['fitbit_access_token'],
    os.environ['fitbit_refresh_token']
)

## Query fitbit data

In [181]:
date_list = pd.date_range('2019-11-24', pd.datetime.today())

food_logs = [auth_client.foods_log(date) for date in date_list]
cal_out = [(date, auth_client.activities(date)['summary']['caloriesOut']) for date in date_list]
macros_data = [[food_log['summary'], date] for food_log, date in zip(food_logs, date_list)]
food_data = []

for food_log in food_logs:
    for item in food_log['foods']:
        food_data.append([item['logDate'], item['loggedFood']['name'], item['loggedFood']['calories'], item['loggedFood']['mealTypeId']])
    

In [182]:
df_calsout = pd.DataFrame(cal_out, columns=['log_date', 'caloriesOut']).set_index('log_date')

In [183]:
# Prepare foods dataframes
df_foods = pd.DataFrame(food_data, columns=['date', 'name', 'calories', 'meal'])

In [184]:
df_top_foods = df_foods.groupby('name').sum().sort_values('calories', ascending=False)[['calories']]
df_top_foods['% of total calories'] = 100*(df_top_foods['calories']/df_foods.sum()['calories'])
df_top_foods['% calories (cumulative)'] = df_top_foods['% of total calories'].cumsum()

In [185]:
df_macros = pd.DataFrame([data[0] for data in macros_data])
df_macros['log_date'] = [data[1] for data in macros_data]
df_macros['carbs(%)'] = 100*(df_macros['carbs']*4)/((df_macros['carbs']*4) + (df_macros['fat']*9) + (df_macros['protein']*4) )
df_macros['fat(%)'] = 100*(df_macros['fat']*9)/((df_macros['carbs']*4) + (df_macros['fat']*9) + (df_macros['protein']*4) )
df_macros['protein(%)'] = 100*(df_macros['protein']*4)/((df_macros['carbs']*4) + (df_macros['fat']*9) + (df_macros['protein']*4) )
df_macros = df_macros.set_index('log_date')
df_macros = df_macros.join(df_calsout)

In [191]:
df_macros['calsIn_over_calsOut'] = df_macros['calories']/df_macros['caloriesOut']
df_macros['carbs_target'] = 350
df_macros['protein_target'] = 200
df_macros['fiber_target'] = 36
df_macros['fat_target'] = 116
df_macros.head()

,calories,carbs,fat,fiber,protein,sodium,water,carbs(%),fat(%),protein(%),caloriesOut,calsIn_over_calsOut,carbs_target,protein_target,fiber_target,fat_target
log_date,,,,,,,,,,,,,,,,
2019-11-24,4267,233.17,305.54,53.76,179.59,3836.37,0.000000,21.192938,62.484037,16.323025,1962,2.174822,350,200,36,116
2019-11-25,3459,313.40,195.93,66.15,159.66,2503.05,142.020004,34.292498,48.237367,17.470135,3437,1.006401,350,200,36,116
2019-11-26,3619,297.81,233.80,65.33,131.44,2302.92,142.020004,31.174500,55.066471,13.759029,3557,1.017430,350,200,36,116
2019-11-27,2905,296.57,98.43,47.65,183.31,5553.59,0.000000,42.285743,31.577428,26.136829,3246,0.894948,350,200,36,116
2019-11-28,2293,292.32,82.54,44.44,70.01,1880.29,0.000000,53.338686,33.886816,12.774498,3805,0.602628,350,200,36,116


In [207]:
def plot_macro(df, macros):
    plot_macros = plotutils.plot_ts(
        df_macros, 
        ys=macros + [f'{macro}_target' for macro in macros], 
        styles=['o-']*len(macros) + ['-']*len(macros),
        date_col='log_date',
        title='Macros (grams)',
        legend_location='top_left'
    )
    show(plot_macros)

In [211]:
plot_macro(df_macros, ['protein', 'carbs'])

In [212]:
plot_macro(df_macros, ['fiber', 'fat'])

Assuming carbs and protein provide `4 cals` per `gram` and fat provides `9 cals` per `gram`, here are the macros percentages

## What do I actually eat...
Let's inspect the food logs since last week of November. Where do most of my calories come from?

In [114]:
print(f"Food by cals:\n{df_foods['date'].min()} to {df_foods['date'].max()}")
df_top_foods.head(50)

Food by cals:
2019-11-24 to 2020-01-19


,calories,% of total calories,% calories (cumulative)
name,,,
"Recovery Formula, Chocolate",17751,9.887980,9.887980
12 Grain Bagels,12650,7.046529,16.934509
Bananas,8978,5.001086,21.935595
Quinoa,8749,4.873525,26.809120
Walnuts,8743,4.870182,31.679302
Salmon Poke Bowl,8160,4.545429,36.224731
Nuttin' Better (Regular),6525,3.634672,39.859404
Vegetarian Poke Bowl,5896,3.284295,43.143699
All Natural Peanut Butter,5640,3.141694,46.285393


In [95]:
df_food_name_mapper = df_top_foods.reset_index()[['name']]
df_food_name_mapper['std_name'] = None
df_food_name_mapper.to_csv('../data/food_name_mapper.csv', index=None)

In [96]:
plotutils.top_food_plot(df_top_foods.head(50))

# Daily food log view

In [117]:
df_foods.tail(50)

,date,name,calories,meal
703,2020-01-16,All Natural Peanut Butter,270,1
704,2020-01-16,12 Grain Bagels,230,1
705,2020-01-16,Apple,55,2
706,2020-01-16,Bananas,840,2
707,2020-01-16,Nuttin' Better (Regular),649,3
708,2020-01-16,Cooked Vegetables,283,5
709,2020-01-16,Thai Peanut Chicken,416,5
710,2020-01-16,Green Curry,347,5
711,2020-01-17,12 Grain Bagels,230,1
712,2020-01-17,"Recovery Formula, Chocolate",500,1
